# Preparación del entorno

In [1]:
import os
import pandas as pd

In [2]:
# Aquí se debe poner la carpeta de la computadora local
carpeta_local = "~/buckets/b1/"

# Establezco el Working Directory
os.chdir(os.path.expanduser(carpeta_local))

# Cargo el dataset
archivo_dataset = "./datasets/competencia_03 (base).csv.gz"
dataset = pd.read_csv(archivo_dataset)


/tmp/ipykernel_3737/717791371.py:9: DtypeWarning: Columns (154) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(archivo_dataset)


# Prueba con SQL

In [3]:
%%bash
pip install --upgrade duckdb
pip install --upgrade jupysql
pip install --upgrade duckdb-engine

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for duckdb from https://files.pythonhosted.org/packages/d3/52/e20f7988add9f9584a9a6917d48dbcb0dae81d5e04132a94da486de69805/duckdb-0.9.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 51.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: duckdb
    Found existing installation: duckdb 0.8.1
    Uninstalling duckdb-0.8.1:
      Successfully uninstalled duckdb-0.8.1
Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for jupysql from https://files.pythonhosted.org/packages/34/ca/4ea8ba339edb13f3cffcf76390b4f35f5f0478c994f254ccf1ae49104241/jupysql-0.10.2-py3-none-any.whl.metadata
  Obtaining dependency information for jupysql-plugin from https://files.pythonhosted.org/packages/34/55/ddd63a65f7c9761f1f7b1d1c98568f5ea69835f8327c19f5e0e90fa0482


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

# Construcción de Clase_ternaria

In [32]:
%%sql
create or replace table competencia_03 as 
select 
    *
from read_csv_auto("./datasets/competencia_03 (base).csv.gz")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [33]:
%sql select * from competencia_03 limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,29183981,201901,1,0,1,48,171,8304.31,42993.54,2501.04,...,8,0.00,-35773.32,2.33,5148,36602.13,54,0,5008.71,CONTINUA
1,29184630,201901,1,0,1,57,296,18318.76,42597.26,14986.76,...,1,0.00,-67414.82,1149.57,3299,28503.06,10,0,1395.87,CONTINUA
2,29185587,201901,1,0,1,77,296,20862.84,85481.31,4228.40,...,1,0.00,0.00,0.00,6790,5170.25,2,0,0.00,CONTINUA
3,29185646,201901,1,0,1,58,231,25218.94,165317.78,4474.25,...,22,69653.59,-59422.00,0.00,7037,67991.90,31,0,2897.31,CONTINUA
4,29187499,201901,1,0,1,69,349,5307.54,32374.08,4706.22,...,8,0.00,-4802.21,0.00,7596,4851.93,3,0,492.66,CONTINUA


In [34]:
%%sql
select
    foto_mes
    , count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes
                           -- y lo guarda en un campo llamado cantidad
from competencia_03
group by foto_mes

,foto_mes,cantidad
0,201901,124864
1,201902,125898
2,201903,126536
3,201904,127059
4,201905,127683
5,201906,129181
6,201907,130551
7,201908,132615
8,201909,134267
9,201910,136515


# Descarga índice de inflación

## Armado del índice

In [35]:
!pip install pandas requests

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [36]:
import requests
from io import BytesIO

In [37]:
# URL del archivo
url = "https://www.indec.gob.ar/ftp/cuadros/economia/sh_ipc_08_23.xls"

# Descargar el archivo y cargar los datos de la hoja correcta
response = requests.get(url)
xls = pd.ExcelFile(BytesIO(response.content))

# Cargar los datos desde la hoja de interés
inflacion = pd.read_excel(xls, sheet_name="Índices IPC Cobertura Nacional", skiprows=5, usecols="B:CC")

In [38]:
inflacion

,2016-12-01,2017-01-01,2017-02-01,2017-03-01,2017-04-01,2017-05-01,2017-06-01,2017-07-01,2017-08-01,2017-09-01,...,2022-10-01,2022-11-01,2022-12-01,2023-01-01,2023-02-01,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100,101.5859,103.6859,106.1476,108.9667,110.5301,111.8477,113.7852,115.3819,117.5719,...,1028.706,1079.2787,1134.5875,1202.979,1282.7091,1381.1601,1497.2147,1613.5895,1709.6115,1818.0838
4,100,101.3024,103.1605,106.0143,108.3515,109.7631,110.7483,112.0115,114.3467,116.4048,...,1096.5069,1135.001,1187.9252,1268.3385,1392.3999,1522.5337,1677.0441,1774.4119,1848.0332,1954.5101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# Transponer los datos y establecer la tercera fila como encabezado
inflacion = inflacion.transpose()


In [40]:
inflacion = inflacion[3]

In [41]:
inflacion

2016-12-01          100
2017-01-01     101.5859
2017-02-01     103.6859
2017-03-01     106.1476
2017-04-01     108.9667
                ...    
2023-03-01    1381.1601
2023-04-01    1497.2147
2023-05-01    1613.5895
2023-06-01    1709.6115
2023-07-01    1818.0838
Name: 3, Length: 80, dtype: object

In [42]:
# Pasar a data frame
import pandas as pd

inflacion = pd.DataFrame(inflacion)

In [43]:
inflacion.index.name='foto_mes'

In [44]:
inflacion

,3
foto_mes,
2016-12-01,100
2017-01-01,101.5859
2017-02-01,103.6859
2017-03-01,106.1476
2017-04-01,108.9667
...,...
2023-03-01,1381.1601
2023-04-01,1497.2147
2023-05-01,1613.5895


In [45]:
# Renombrar la columna
inflacion.columns = ["indice_inflacion"]

# Eliminar las filas con valores nulos
inflacion.dropna(inplace=True)

# Eliminar guiones y los últimos dos dígitos de la columna "foto_mes"
inflacion.index = inflacion.index.astype(str).str.replace('-', '').str[:-2]

# Mostrar las primeras filas del DataFrame resultante
inflacion.head()

,indice_inflacion
foto_mes,
201612,100
201701,101.5859
201702,103.6859
201703,106.1476
201704,108.9667


## Inflación A Sql

In [46]:
# Guardar el DataFrame como un archivo CSV
inflacion.to_csv('inflacion.csv')

In [47]:
%%sql
create or replace table inflacion as
select
    *
from read_csv_auto('inflacion.csv')

,Success


In [48]:
%%sql
select *
from inflacion

,foto_mes,indice_inflacion
0,201612,100.0000
1,201701,101.5859
2,201702,103.6859
3,201703,106.1476
4,201704,108.9667
...,...,...
75,202303,1381.1601
76,202304,1497.2147
77,202305,1613.5895
78,202306,1709.6115


Agrego inflación acumulada hasta 202107 para hacer el drifting

In [49]:
%%sql
ALTER TABLE inflacion
ADD COLUMN indice_inflacion_acumulada FLOAT; -- Adjust the data type as needed

WITH InflacionAcumulada AS (
    SELECT
        foto_mes,
        indice_inflacion,
        (
            (SELECT indice_inflacion FROM inflacion WHERE foto_mes = '202107') / indice_inflacion
        ) AS inflacion_acumulada
    FROM
        inflacion
)
UPDATE inflacion
SET indice_inflacion_acumulada = (SELECT inflacion_acumulada FROM InflacionAcumulada WHERE inflacion.foto_mes = InflacionAcumulada.foto_mes);


,Count
0,80


In [50]:
%%sql
select *
from inflacion

,foto_mes,indice_inflacion,indice_inflacion_acumulada
0,201612,100.0000,4.980987
1,201701,101.5859,4.903227
2,201702,103.6859,4.803919
3,201703,106.1476,4.692510
4,201704,108.9667,4.571109
...,...,...,...
75,202303,1381.1601,0.360638
76,202304,1497.2147,0.332684
77,202305,1613.5895,0.308690
78,202306,1709.6115,0.291352


# Inner join competencia_03 e inflación

In [51]:
%%sql
CREATE TABLE competencia_03_temp AS
SELECT
    *
FROM
    competencia_03 c2
INNER JOIN
    inflacion i
ON
    c2.foto_mes = i.foto_mes;

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [52]:
%%sql
-- Elimina la columna 'indice_inflacion' de la tabla temporal
ALTER TABLE competencia_03_temp
DROP COLUMN indice_inflacion;

-- Borra la tabla original 'competencia_03'
DROP TABLE competencia_03;

-- Renombra la tabla temporal a 'competencia_03'
ALTER TABLE competencia_03_temp
RENAME TO competencia_03;

,Success


In [53]:
%%sql
SELECT
    *
FROM
    competencia_03 c2

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria,foto_mes:1,indice_inflacion_acumulada
0,29183981,201901,1,0,1,48,171,8304.31,42993.54,2501.04,...,-35773.32,2.33,5148.0,36602.13,54.0,0.0,5008.71,CONTINUA,201901,2.626963
1,29184630,201901,1,0,1,57,296,18318.76,42597.26,14986.76,...,-67414.82,1149.57,3299.0,28503.06,10.0,0.0,1395.87,CONTINUA,201901,2.626963
2,29185587,201901,1,0,1,77,296,20862.84,85481.31,4228.40,...,0.00,0.00,6790.0,5170.25,2.0,0.0,0.00,CONTINUA,201901,2.626963
3,29185646,201901,1,0,1,58,231,25218.94,165317.78,4474.25,...,-59422.00,0.00,7037.0,67991.90,31.0,0.0,2897.31,CONTINUA,201901,2.626963
4,29187499,201901,1,0,1,69,349,5307.54,32374.08,4706.22,...,-4802.21,0.00,7596.0,4851.93,3.0,0.0,492.66,CONTINUA,201901,2.626963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893023,188116410,202109,1,0,0,31,1,53.80,53.80,0.00,...,0.00,0.00,23.0,807.91,2.0,0.0,35.19,None,202109,0.942482
4893024,188117144,202109,0,0,0,31,1,0.00,0.00,0.00,...,NaN,NaN,11.0,NaN,NaN,NaN,0.00,None,202109,0.942482
4893025,188119945,202109,0,0,0,51,1,0.53,0.53,0.00,...,NaN,NaN,7.0,NaN,NaN,NaN,0.00,None,202109,0.942482
4893026,188127016,202109,0,0,0,31,1,0.00,0.00,0.00,...,NaN,NaN,8.0,NaN,NaN,NaN,0.00,None,202109,0.942482


# Solución del Data Drifting a variables monetarias

Primero, ajustos todas las variables monetarias por la inflación acumulada a 202107

In [54]:
%%sql
ALTER TABLE competencia_03
ADD COLUMN Mrentabilidad_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mrentabilidad_annual_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcomisiones_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mactivos_margen_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mpasivos_margen_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcuenta_corriente_adicional_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcuenta_corriente_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcaja_ahorro_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcaja_ahorro_adicional_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcaja_ahorro_dolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcuentas_saldo_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mautoservicio_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mtarjeta_visa_consumo_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mtarjeta_master_consumo_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mprestamos_personales_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mprestamos_prendarios_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mprestamos_hipotecarios_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mplazo_fijo_dolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mplazo_fijo_pesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN minversion1_pesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN minversion1_dolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN minversion2_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mpayroll_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mpayroll2_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcuenta_debitos_automaticos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mttarjeta_master_debitos_automaticos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mpagodeservicios_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mpagomiscuentas_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcajeros_propios_descuentos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mtarjeta_visa_descuentos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mtarjeta_master_descuentos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcomisiones_mantenimiento_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcomisiones_otras_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mforex_buy_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mforex_sell_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mtransferencias_recibidas_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mtransferencias_emitidas_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mextraccion_autoservicio_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcheques_depositados_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcheques_emitidos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcheques_depositados_rechazados_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcheques_emitidos_rechazados_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN matm_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN matm_other_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mfinanciacion_limite_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_msaldototal_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_msaldopesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_msaldodolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mconsumospesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mconsumosdolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mlimitecompra_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_madelantopesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_madelantodolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mpagado_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mpagospesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mpagosdolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mconsumototal_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mpagominimo_ajustado DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mfinanciacion_limite_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_msaldototal_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_msaldopesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_msaldodolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mconsumospesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mconsumosdolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mlimitecompra_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_madelantopesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_madelantodolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mpagado_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mpagospesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mpagosdolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mconsumototal_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mpagominimo_ajustado DECIMAL(18, 2);

,Success


In [55]:
%%sql
UPDATE competencia_03
SET
    Mrentabilidad_ajustada = Mrentabilidad * indice_inflacion_acumulada,
    mrentabilidad_annual_ajustada = mrentabilidad_annual * indice_inflacion_acumulada,
    mcomisiones_ajustada = mcomisiones * indice_inflacion_acumulada,
    mactivos_margen_ajustada = mactivos_margen * indice_inflacion_acumulada,
    mpasivos_margen_ajustada = mpasivos_margen * indice_inflacion_acumulada,
    mcuenta_corriente_adicional_ajustada = mcuenta_corriente_adicional * indice_inflacion_acumulada,
    mcuenta_corriente_ajustada = mcuenta_corriente * indice_inflacion_acumulada,
    mcaja_ahorro_ajustada = mcaja_ahorro * indice_inflacion_acumulada,
    mcaja_ahorro_adicional_ajustada = mcaja_ahorro_adicional * indice_inflacion_acumulada,
    mcaja_ahorro_dolares_ajustada = mcaja_ahorro_dolares * indice_inflacion_acumulada,
    mcuentas_saldo_ajustada = mcuentas_saldo * indice_inflacion_acumulada,
    mautoservicio_ajustada = mautoservicio * indice_inflacion_acumulada,
    mtarjeta_visa_consumo_ajustada = mtarjeta_visa_consumo * indice_inflacion_acumulada,
    mtarjeta_master_consumo_ajustada = mtarjeta_master_consumo * indice_inflacion_acumulada,
    mprestamos_personales_ajustada = mprestamos_personales * indice_inflacion_acumulada,
    mprestamos_prendarios_ajustada = mprestamos_prendarios * indice_inflacion_acumulada,
    mprestamos_hipotecarios_ajustada = mprestamos_hipotecarios * indice_inflacion_acumulada,
    mplazo_fijo_dolares_ajustada = mplazo_fijo_dolares * indice_inflacion_acumulada,
    mplazo_fijo_pesos_ajustada = mplazo_fijo_pesos * indice_inflacion_acumulada,
    minversion1_pesos_ajustada = minversion1_pesos * indice_inflacion_acumulada,
    minversion1_dolares_ajustada = minversion1_dolares * indice_inflacion_acumulada,
    minversion2_ajustada = minversion2 * indice_inflacion_acumulada,
    mpayroll_ajustada = mpayroll * indice_inflacion_acumulada,
    mpayroll2_ajustada = mpayroll2 * indice_inflacion_acumulada,
    mcuenta_debitos_automaticos_ajustada = mcuenta_debitos_automaticos * indice_inflacion_acumulada,
    mttarjeta_master_debitos_automaticos_ajustada = mttarjeta_master_debitos_automaticos * indice_inflacion_acumulada,
    mpagodeservicios_ajustada = mpagodeservicios * indice_inflacion_acumulada,
    mpagomiscuentas_ajustada = mpagomiscuentas * indice_inflacion_acumulada,
    mcajeros_propios_descuentos_ajustada = mcajeros_propios_descuentos * indice_inflacion_acumulada,
    mtarjeta_visa_descuentos_ajustada = mtarjeta_visa_descuentos * indice_inflacion_acumulada,
    mtarjeta_master_descuentos_ajustada = mtarjeta_master_descuentos * indice_inflacion_acumulada,
    mcomisiones_mantenimiento_ajustada = mcomisiones_mantenimiento * indice_inflacion_acumulada,
    mcomisiones_otras_ajustada = mcomisiones_otras * indice_inflacion_acumulada,
    mforex_buy_ajustada = mforex_buy * indice_inflacion_acumulada,
    mforex_sell_ajustada = mforex_sell * indice_inflacion_acumulada,
    mtransferencias_recibidas_ajustada = mtransferencias_recibidas * indice_inflacion_acumulada,
    mtransferencias_emitidas_ajustada = mtransferencias_emitidas * indice_inflacion_acumulada,
    mextraccion_autoservicio_ajustada = mextraccion_autoservicio * indice_inflacion_acumulada,
    mcheques_depositados_ajustada = mcheques_depositados * indice_inflacion_acumulada,
    mcheques_emitidos_ajustada = mcheques_emitidos * indice_inflacion_acumulada,
    mcheques_depositados_rechazados_ajustada = mcheques_depositados_rechazados * indice_inflacion_acumulada,
    mcheques_emitidos_rechazados_ajustada = mcheques_emitidos_rechazados * indice_inflacion_acumulada,
    matm_ajustada = matm * indice_inflacion_acumulada,
    matm_other_ajustada = matm_other * indice_inflacion_acumulada,
    Master_mfinanciacion_limite_ajustada = Master_mfinanciacion_limite * indice_inflacion_acumulada,
    Master_msaldototal_ajustada = Master_msaldototal * indice_inflacion_acumulada,
    Master_msaldopesos_ajustada = Master_msaldopesos * indice_inflacion_acumulada,
    Master_msaldodolares_ajustada = Master_msaldodolares * indice_inflacion_acumulada,
    Master_mconsumospesos_ajustada = Master_mconsumospesos * indice_inflacion_acumulada,
    Master_mconsumosdolares_ajustada = Master_mconsumosdolares * indice_inflacion_acumulada,
    Master_mlimitecompra_ajustada = Master_mlimitecompra * indice_inflacion_acumulada,
    Master_madelantopesos_ajustada = Master_madelantopesos * indice_inflacion_acumulada,
    Master_madelantodolares_ajustada = Master_madelantodolares * indice_inflacion_acumulada,
    Master_mpagado_ajustada = Master_mpagado * indice_inflacion_acumulada,
    Master_mpagospesos_ajustada = Master_mpagospesos * indice_inflacion_acumulada,
    Master_mpagosdolares_ajustada = Master_mpagosdolares * indice_inflacion_acumulada,
    Master_mconsumototal_ajustada = Master_mconsumototal * indice_inflacion_acumulada,
    Master_mpagominimo_ajustado = Master_mpagominimo * indice_inflacion_acumulada,
    Visa_mfinanciacion_limite_ajustada = Visa_mfinanciacion_limite * indice_inflacion_acumulada,
    Visa_msaldototal_ajustada = Visa_msaldototal * indice_inflacion_acumulada,
    Visa_msaldopesos_ajustada = Visa_msaldopesos * indice_inflacion_acumulada,
    Visa_msaldodolares_ajustada = Visa_msaldodolares * indice_inflacion_acumulada,
    Visa_mconsumospesos_ajustada = Visa_mconsumospesos * indice_inflacion_acumulada,
    Visa_mconsumosdolares_ajustada = Visa_mconsumosdolares * indice_inflacion_acumulada,
    Visa_mlimitecompra_ajustada = Visa_mlimitecompra * indice_inflacion_acumulada,
    Visa_madelantopesos_ajustada = Visa_madelantopesos * indice_inflacion_acumulada,
    Visa_madelantodolares_ajustada = Visa_madelantodolares * indice_inflacion_acumulada,
    Visa_mpagado_ajustada = Visa_mpagado * indice_inflacion_acumulada,
    Visa_mpagospesos_ajustada = Visa_mpagospesos * indice_inflacion_acumulada,
    Visa_mpagosdolares_ajustada = Visa_mpagosdolares * indice_inflacion_acumulada,
    Visa_mconsumototal_ajustada = Visa_mconsumototal * indice_inflacion_acumulada,
    Visa_mpagominimo_ajustado = Visa_mpagominimo * indice_inflacion_acumulada;


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


Después,ya elimino la variable "inflación_acumulada" ya que no es de más utilidad, y las variables monetarias nominales

In [56]:
%%sql
ALTER TABLE competencia_03
DROP COLUMN indice_inflacion_acumulada;

,Success


In [57]:
%%sql
ALTER TABLE competencia_03
DROP COLUMN Mrentabilidad;

ALTER TABLE competencia_03
DROP COLUMN mrentabilidad_annual;

ALTER TABLE competencia_03
DROP COLUMN mcomisiones;

ALTER TABLE competencia_03
DROP COLUMN mactivos_margen;

ALTER TABLE competencia_03
DROP COLUMN mpasivos_margen;

ALTER TABLE competencia_03
DROP COLUMN mcuenta_corriente_adicional;

ALTER TABLE competencia_03
DROP COLUMN mcuenta_corriente;

ALTER TABLE competencia_03
DROP COLUMN mcaja_ahorro;

ALTER TABLE competencia_03
DROP COLUMN mcaja_ahorro_adicional;

ALTER TABLE competencia_03
DROP COLUMN mcaja_ahorro_dolares;

ALTER TABLE competencia_03
DROP COLUMN mcuentas_saldo;

ALTER TABLE competencia_03
DROP COLUMN mautoservicio;

ALTER TABLE competencia_03
DROP COLUMN mtarjeta_visa_consumo;

ALTER TABLE competencia_03
DROP COLUMN mtarjeta_master_consumo;

ALTER TABLE competencia_03
DROP COLUMN mprestamos_personales;

ALTER TABLE competencia_03
DROP COLUMN mprestamos_prendarios;

ALTER TABLE competencia_03
DROP COLUMN mprestamos_hipotecarios;

ALTER TABLE competencia_03
DROP COLUMN mplazo_fijo_dolares;

ALTER TABLE competencia_03
DROP COLUMN mplazo_fijo_pesos;

ALTER TABLE competencia_03
DROP COLUMN minversion1_pesos;

ALTER TABLE competencia_03
DROP COLUMN minversion1_dolares;

ALTER TABLE competencia_03
DROP COLUMN minversion2;

ALTER TABLE competencia_03
DROP COLUMN mpayroll;

ALTER TABLE competencia_03
DROP COLUMN mpayroll2;

ALTER TABLE competencia_03
DROP COLUMN mcuenta_debitos_automaticos;

--ALTER TABLE competencia_03
--DROP COLUMN mtarjeta_visa_debitos_automaticos;

ALTER TABLE competencia_03
DROP COLUMN mttarjeta_master_debitos_automaticos;

ALTER TABLE competencia_03
DROP COLUMN mpagodeservicios;

ALTER TABLE competencia_03
DROP COLUMN mpagomiscuentas;

ALTER TABLE competencia_03
DROP COLUMN mcajeros_propios_descuentos;

ALTER TABLE competencia_03
DROP COLUMN mtarjeta_visa_descuentos;

ALTER TABLE competencia_03
DROP COLUMN mtarjeta_master_descuentos;

ALTER TABLE competencia_03
DROP COLUMN mcomisiones_mantenimiento;

ALTER TABLE competencia_03
DROP COLUMN mcomisiones_otras;

ALTER TABLE competencia_03
DROP COLUMN mforex_buy;

ALTER TABLE competencia_03
DROP COLUMN mforex_sell;

ALTER TABLE competencia_03
DROP COLUMN mtransferencias_recibidas;

ALTER TABLE competencia_03
DROP COLUMN mtransferencias_emitidas;

ALTER TABLE competencia_03
DROP COLUMN mextraccion_autoservicio;

ALTER TABLE competencia_03
DROP COLUMN mcheques_depositados;

ALTER TABLE competencia_03
DROP COLUMN mcheques_emitidos;

ALTER TABLE competencia_03
DROP COLUMN mcheques_depositados_rechazados;

ALTER TABLE competencia_03
DROP COLUMN mcheques_emitidos_rechazados;

ALTER TABLE competencia_03
DROP COLUMN matm;

ALTER TABLE competencia_03
DROP COLUMN Master_mfinanciacion_limite;

ALTER TABLE competencia_03
DROP COLUMN Master_msaldototal;

ALTER TABLE competencia_03
DROP COLUMN Master_msaldopesos;

ALTER TABLE competencia_03
DROP COLUMN Master_msaldodolares;

ALTER TABLE competencia_03
DROP COLUMN Master_mconsumospesos;

ALTER TABLE competencia_03
DROP COLUMN Master_mconsumosdolares;

ALTER TABLE competencia_03
DROP COLUMN Master_mlimitecompra;

ALTER TABLE competencia_03
DROP COLUMN Master_madelantopesos;

ALTER TABLE competencia_03
DROP COLUMN Master_madelantodolares;

ALTER TABLE competencia_03
DROP COLUMN Master_mpagado;

ALTER TABLE competencia_03
DROP COLUMN Master_mpagospesos;

ALTER TABLE competencia_03
DROP COLUMN Master_mpagosdolares;

ALTER TABLE competencia_03
DROP COLUMN Master_mconsumototal;

ALTER TABLE competencia_03
DROP COLUMN Master_mpagominimo;

ALTER TABLE competencia_03
DROP COLUMN Visa_mfinanciacion_limite;

ALTER TABLE competencia_03
DROP COLUMN Visa_msaldototal;

ALTER TABLE competencia_03
DROP COLUMN Visa_msaldopesos;

ALTER TABLE competencia_03
DROP COLUMN Visa_msaldodolares;

ALTER TABLE competencia_03
DROP COLUMN Visa_mconsumospesos;

ALTER TABLE competencia_03
DROP COLUMN Visa_mconsumosdolares;

ALTER TABLE competencia_03
DROP COLUMN Visa_mlimitecompra;

ALTER TABLE competencia_03
DROP COLUMN Visa_madelantopesos;

ALTER TABLE competencia_03
DROP COLUMN Visa_madelantodolares;

ALTER TABLE competencia_03
DROP COLUMN Visa_mpagado;

ALTER TABLE competencia_03
DROP COLUMN Visa_mpagospesos;

ALTER TABLE competencia_03
DROP COLUMN Visa_mpagosdolares;

ALTER TABLE competencia_03
DROP COLUMN Visa_mconsumototal;

ALTER TABLE competencia_03
DROP COLUMN Visa_mpagominimo;


,Success


In [61]:
%%sql
select * from competencia_03 limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,cproductos,tcuentas,ccuenta_corriente,...,Visa_mconsumospesos_ajustada,Visa_mconsumosdolares_ajustada,Visa_mlimitecompra_ajustada,Visa_madelantopesos_ajustada,Visa_madelantodolares_ajustada,Visa_mpagado_ajustada,Visa_mpagospesos_ajustada,Visa_mpagosdolares_ajustada,Visa_mconsumototal_ajustada,Visa_mpagominimo_ajustado
0,29183981,201901,1,0,1,48,171,9,1,1,...,96152.44,65.46,1584624.08,0.0,0.0,0.0,-93975.18,6.12,96152.44,13157.70
1,29184630,201901,1,0,1,57,296,9,1,1,...,74876.48,198.31,1203744.78,0.0,0.0,0.0,-177096.23,3019.88,74876.48,3666.90
2,29185587,201901,1,0,1,77,296,11,1,1,...,13582.05,0.00,1569159.15,0.0,0.0,0.0,0.00,0.00,13582.05,0.00
3,29185646,201901,1,0,1,58,231,9,1,1,...,178612.20,20.33,1571065.80,0.0,0.0,182977.4,-156099.39,0.00,178612.20,7611.13
4,29187499,201901,1,0,1,69,349,10,1,1,...,12745.84,0.00,73954.26,0.0,0.0,0.0,-12615.23,0.00,12745.84,1294.20


# Guardar el csv con las variables transformadas

In [62]:
%%sql
copy competencia_03 to './datasets/competencia_03_infla.csv.gz' (FORMAT CSV, HEADER)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success
